script

In [ ]:
import time
import RPi.GPIO as GPIO
from adafruit_pca9685 import PCA9685
from board import SCL, SDA
import busio

# Minimum and maximum pulse length out of 4096 (12-bit)
# These values may need calibration depending on your servo
SERVO_MIN = 150  # Min pulse length out of 4096 (approx 0.5 ms pulse)
SERVO_MAX = 600  # Max pulse length out of 4096 (approx 2.5 ms pulse)
ESC_PIN = 4

# Setup GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(ESC_PIN, GPIO.OUT)
# Setup PWM at 50Hz
pwm_esc = GPIO.PWM(ESC_PIN, 50)
pwm_esc.start(0)

def angle_to_pwm(angle):
    # Convert angle (0-180) to PWM pulse length for PCA9685
    pulse_span = SERVO_MAX - SERVO_MIN
    pwm_value = int(SERVO_MIN + (pulse_span * angle / 180))
    return pwm_value

def set_servo_angle(pca, channel, angle):
    pwm_value = angle_to_pwm(180)
    pca.channels[channel].duty_cycle = pwm_value * 16
    time.sleep(angle * 1.5/180)
    pca.channels[channel].duty_cycle =0 *16# duty_cycle is 16-bit (0-65535), pulse is 12-bit (0-4095)
    print(f"Moved servo to {angle} degrees")  # duty_cycle is 16-bit (0-65535), pulse is 12-bit (0-4095)

def set_servo_angle2(pca, channel, angle):
    pwm_value = angle_to_pwm(20)
    pca.channels[channel].duty_cycle = pwm_value * 16
    time.sleep(angle * 1.5/180)
    pca.channels[channel].duty_cycle =0 *16# duty_cycle is 16-bit (0-65535), pulse is 12-bit (0-4095)
    print(f"Moved servo to {angle} degrees")  # duty_cycle is 16-bit (0-65535), pulse is 12-bit (0-4095)

def set_servo_angle3(pca, channel, angle):
    pwm_value = angle_to_pwm(-20)
    pca.channels[channel].duty_cycle = pwm_value * 16
    time.sleep(angle * 1.5/180)
    pca.channels[channel].duty_cycle =0 *16# duty_cycle is 16-bit (0-65535), pulse is 12-bit (0-4095)
    print(f"Moved servo to {angle} degrees")

def move_up(pca):
    set_servo_angle2(pca, 1, 60)
    time.sleep(1)
    set_servo_angle(pca, 2, 100)
    time.sleep(1)
    set_servo_angle2(pca, 3, 80)
    time.sleep(1)
    set_servo_angle(pca, 4, 90)
    time.sleep(1)

def move_down(pca):
    set_servo_angle2(pca, 4, 60)
    time.sleep(1)
    set_servo_angle(pca, 3, 60)
    time.sleep(1)
    set_servo_angle2(pca, 2, 50)
    time.sleep(1)
    set_servo_angle3(pca, 1, 60)
    time.sleep(1)
    #need testing

def move_left(pca):
    set_servo_angle(pca, 0, 90)
    time.sleep(1)

def move_right(pca):
    set_servo_angle2(pca, 0, 90)
    time.sleep(1)
    #need testing

def main():
    # Initialize I2C bus and PCA9685 module
    i2c = busio.I2C(SCL, SDA)
    pca = PCA9685(i2c)
    pca.frequency = 50  # Typical servo frequency is 50 Hz
    channel_index = 0  # Using channel 0 by default; change if your servo is on another channel

    try:
        while True:
            # Script
            pwm_esc.ChangeDutyCycle(8)
            time .sleep(0.5)
            pwm_esc.ChangeDutyCycle(0)

            move_up(pca)
            move_left(pca)
            move_down(pca)
            move_right(pca)
            time.sleep(10)
            pwm_esc.ChangeDutyCycle(8)
            time.sleep(2)

            pwm_esc.ChangeDutyCycle(0)
            move_right(pca)
            move_up(pca)
            move_down(pca)
            move_left(pca)
            time.sleep(2)
            pwm_esc.ChangeDutyCycle(8)
            time.sleep(2)

            pwm_esc.ChangeDutyCycle(0)
            pwm_esc.ChangeDutyCycle(5.8)
            time.sleep(12)
            pwm_esc.ChangeDutyCycle(0)
            print("end")
    except KeyboardInterrupt:
        print("\nExiting")

    finally:
        pca.deinit()
        print("PCA9685 deinitialized")

if __name__ == "__main__":
    main()

interface

In [ ]:
from flask import Flask, render_template_string, Response, jsonify, request
import psutil
import netifaces
import cv2
import os
from ultralytics import YOLO# Load the YOLO model
try:
    model = YOLO('sa.pt')
    class_list = model.names
except Exception as e:
    print(f"Error loading model: {e}")
    exit()


app = Flask(__name__)

# Initial mode (default to automatic)
MODE = 'automatique'

# Template Jinja2html>

TEMPLATE = """
<!doctype html>
<html lang="fr">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Robot Dashboard Ultimate</title>
  <!-- Darkly Bootswatch theme and FontAwesome -->
  <link href="https://cdn.jsdelivr.net/npm/bootswatch@5.3.0/dist/darkly/bootstrap.min.css" rel="stylesheet">
  <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css" rel="stylesheet">
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
  <style>
    html, body {
      height: 100%;
      margin: 0;
      padding: 0;
      background-color: #121212;
      color: #fff;
      font-family: 'Arial', sans-serif;
    }

    /* Navbar */
    nav {
      background-color: #222;
      padding: 1rem;
    }

    .navbar-brand {
      font-size: 1.5rem;
      font-weight: bold;
    }

    .btn-sm {
      margin-right: 10px;
    }

    /* Upper part (Metrics) */
    .upper {
      display: flex;
      justify-content: space-between;
      align-items: stretch;
      height: 35vh; /* The upper part occupies 35% of the screen height */
      padding: 1rem;
      gap: 1rem;
    }

    .upper .card {
      flex: 1;
      background-color: #1f1f1f;
      border: 2px solid #bb162b;
      display: flex;
      flex-direction: column;
      color: #ffffff;
      transition: transform 0.3s ease, box-shadow 0.3s ease;
    }

    .card:hover {
      transform: scale(1.05);
      box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3);
    }

    .card-header {
      background-color: #bb162b;
      font-size: 1.25rem;
      display: flex;
      align-items: center;
      gap: 0.5rem;
      padding: 0.75rem;
    }

    .card-body {
      flex: 1;
      display: flex;
      justify-content: center;
      align-items: center;
    }
/* Lower part (Camera feed) */
    .lower {
      height: calc(100vh - 35vh); /* Ensure the lower part takes the remaining space */
      display: flex;
      justify-content: center;
      align-items: center;
      padding: 1rem;
    }

    .lower img {
      width: 100%;
      height: 100%;
      object-fit: cover; /* Ensure the video fills the container */
      border-top: 4px solid #bb162b;
      border-radius: 15px;
    }

    #batteryGauge, #cpuChart {
      height: 180px !important;
    }

    /* Buttons */
    .btn {
      font-size: 0.9rem;
      padding: 8px 16px;
      background-color: #bb162b;
      border: none;
      color: white;
      border-radius: 25px;
      transition: background-color 0.3s;
    }

    .btn:hover {
      background-color: #e03a3e;
    }

    /* Animation for Clock */
    @keyframes pulse {
      0% {
        opacity: 0.6;
      }
      50% {
        opacity: 1;
      }
      100% {
        opacity: 0.6;
      }
    }

    #clock {
      animation: pulse 1s infinite;
    }

  </style>
</head>
<body>
  <nav class="navbar navbar-dark bg-secondary">
    <div class="container-fluid">
      <span class="navbar-brand mb-0 h1"><i class="fas fa-robot"></i> Robot Dashboard</span>
      <div class="d-flex align-items-center text-white">
        <span class="me-3"><i class="far fa-clock"></i> <span id="clock"></span></span>
        <form method="POST" action="/set_mode/automatique" style="display:inline">
          <button type="submit" class="btn btn-outline-light btn-sm {{ 'active' if mode=='automatique' else '' }}">Auto</button>
        </form>
        <form method="POST" action="/set_mode/teleop" style="display:inline">
          <button type="submit" class="btn btn-outline-light btn-sm {{ 'active' if mode=='teleop' else '' }}">Teleop</button>
        </form>
      </div>
    </div>
  </nav>

  <div class="upper">
    <div class="card text-center shadow">
      <div class="card-header"><i class="fas fa-battery-three-quarters"></i> Batterie</div>
      <div class="card-body"><canvas id="batteryGauge"></canvas></div>
    </div>
    <div class="card text-center shadow">
      <div class="card-header"><i class="fas fa-microchip"></i> CPU Usage</div>
      <div class="card-body"><canvas id="cpuChart"></canvas></div>
    </div>
    <div class="card text-center shadow">
      <div class="card-header"><i class="fas fa-info-circle"></i> Info Robot</div>
      <div class="card-body">
        <div>
          <p><strong>IP:</strong> {{ ip }}</p>
          <p><strong>Mode:</strong> {{ mode.capitalize() }}</p>
        </div>
      </div>
    </div>
  </div>

  <div class="lower">
    <img src="{{ url_for('video_feed') }}" alt="Flux CamÃ©ra" class="shadow">
  </div>

<script>
  // Clock update
  function updateClock(){
    document.getElementById('clock').textContent = new Date().toLocaleTimeString();
  }
  setInterval(updateClock, 1000);
  updateClock();

  // Battery gauge (red accent)
  const batCtx = document.getElementById('batteryGauge').getContext('2d');
  new Chart(batCtx, {
    type: 'doughnut',
    data: {
      labels: ['ChargÃ©','Libre'],
      datasets:[{ data:[{{ battery }},100-{{ battery }}],
        backgroundColor:['#bb162b','#444'], hoverBackgroundColor:['#e03a3e','#666'], borderWidth:0 }]
    },
    options:{
      rotation:-90*Math.PI/180,
      circumference:180*Math.PI/180,
      cutout:'70%',
      plugins:{ legend:{display:false},
        title:{display:true,text:'{{ battery }}%',color:'#ffffff',font:{size:24}}}
    }
  });

  // CPU chart line in red
  const cpuCtx = document.getElementById('cpuChart').getContext('2d');
  const cpuData = { labels:[], datasets:[{ data:[], label:'CPU %', borderColor:'#bb162b', backgroundColor:'rgba(187,22,43,0.2)', tension:0.3 }] };
  const cpuChart = new Chart(cpuCtx,{
    type:'line', data:cpuData, options:{
      scales:{ x:{ display:false }, y:{ min:0, max:100 } },
      plugins:{ legend:{ display:false } }
    }
  });
  setInterval(()=>{
    const t=new Date().toLocaleTimeString();
    if(cpuData.labels.length>20){ cpuData.labels.shift(); cpuData.datasets[0].data.shift(); }
    cpuData.labels.push(t); cpuData.datasets[0].data.push(Math.floor(Math.random()*50+10));
    cpuChart.update();
  },1000);
</script>
</body>
</html>


"""


# Backend Flask Logic
import psutil
import netifaces
import cv2
from flask import Flask, render_template_string, Response, jsonify, request
import os

app = Flask(__name__)

# Get real-time data for the robot
def get_battery():
    try:
        battery = psutil.sensors_battery()
        return battery.percent if battery else 0
    except:
        return 0

def get_ip():
    for iface in ['wlan0', 'eth0']:
        try:
            addrs = netifaces.ifaddresses(iface).get(netifaces.AF_INET)
            if addrs:
                return addrs[0]['addr']
        except:
            pass
    return "0.0.0.0"

def get_cpu_usage():
    try:
        return psutil.cpu_percent(interval=1)
    except:
        return 0

# Route to display the dashboard
@app.route('/')
def index():
    data = {
        'battery': 67,
        'ip': get_ip(),
        'mode': "automatique"  # Default mode
    }
    return render_template_string(TEMPLATE, **data)

# Route for metrics in JSON format
@app.route('/metrics')
def metrics():
    return jsonify({
        'battery': get_battery(),
        'cpu': get_cpu_usage(),
        'ip': get_ip()
    })

# Route to change the mode of the robot
@app.route('/set_mode/<mode>', methods=['POST'])
def set_mode(mode):
    if mode in ['automatique', 'teleop']:
        # Here you could set a global variable or take some action
        return jsonify({"message": f"Mode changed to {mode}"}), 200
    return jsonify({"message": "Invalid mode"}), 400

# Streaming video feed from the camera
@app.route('/video_feed')
def video_feed():
    def gen():
        cap = cv2.VideoCapture(0)
        # Set camera resolution
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            #cv2.imshow("im",frame)
            results = model.track(frame, persist=True)

            if results[0].boxes.data is not None:
                boxes = results[0].boxes.xyxy
                track_ids = results[0].boxes.id.int() if results[0].boxes.id is not None else []
                class_indices = results[0].boxes.cls.int()
                confidences = results[0].boxes.conf

                for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
                    x1, y1, x2, y2 = map(int, box)
                    class_name = class_list[int(class_idx)]

            # Add text with class name and ID
                    cv2.putText(frame, f"ID: {class_name}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            # Draw the detection box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            ret, buf = cv2.imencode('.jpg', frame)
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + buf.tobytes() + b'\r\n')
        cap.release()

    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

# Run the app
if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port,threaded=True)